In [412]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

# Models

In [379]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

In [380]:
import pandas as pd

In [381]:
df = pd.DataFrame(request.items(), columns=['name', 'qty'])
df

,name,qty
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1


In [382]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat')


In [383]:
from app.models import *
from app.models import Boiling as BoilingModel

In [384]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [385]:
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())

In [386]:
df

,name,qty,sku
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>


In [387]:
sku = df.iloc[0]['sku']

In [388]:
session.query(BoilingModel).first()

<Boiling 1>

In [389]:
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())

In [390]:
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [391]:
lines = ['cheese_maker', 'melting', 'termizator', 'packing']

# Block

In [414]:
# ROWS = {'water_1': 6, 'water_2': 9, 'salt_1': 15, 'salt_2': 18, 'water_melting': 24, 'water_packing': 29, 'salt_melting': 33, 'salt_packing': 46}
ROWS = {'water_1': 1, 'water_2': 3, 'water_melting': 5, 'water_packing': 8}

def cast_row(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        return ROWS[obj]
    else:
        raise Exception('Unknown object type')

In [415]:
style = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00', 'y': cast_row('water_melting')},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00', 'y': cast_row('water_packing')},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'}
}

# Advanced Validator

In [416]:
from utils_ak.clock import clock, clockify

In [417]:
@clockify()
def atom_validate(b1, b2):
    b1, b2 = list(sorted((b1, b2), key=lambda b: [b['class'], b.interval_str]))
        
    if b1['class'] == b2['class'] == 'termizator':
        validate_disjoint(b1, b2)
        
    if b1.get('line') == b2.get('line'):
        if {b1['class'], b2['class']} in [{'pouring'}, {'melting'}, {'packing'}]:
            validate_disjoint(b1, b2)
        
        if {b1['class'], b2['class']} in [{'packing', 'cooling2'}]:
            assert b1.interval.upper <= b2.interval.lower

    
@clockify()
def atom_validator(b1, b2):
    for atom_b1 in b1:
        for atom_b2 in b2:
            atom_validate(atom_b1, atom_b2)    

In [418]:
clock.reset()

In [419]:
root = make('root', t=10).block

with make('pouring', size=5, line=1):
    make('termizator', size=1)
with make('pouring', size=5, line=2):
    make('termizator', size=1)
    
for block in fetch_constructed_blocks():
    root.push(block, validator=atom_validator)
root

root None
  pouring (10,15]
    termizator (10,11]
  pouring (11,16]
    termizator (11,12]

In [429]:
load('src/time.py', import_globals=True, reload=True)
load('src/block.py', import_globals=True, reload=True)
load('src/color.py', import_globals=True, reload=True)
load('src/drawing.py', import_globals=True, reload=True)

def make_boiling(LINE, BID):
    with make('boiling', block_num=12, line=LINE, boiling_id=BID) as boiling:
        total_size = 27
        with make('pouring', y=cast_row(f'water_{LINE}'), size=total_size):
            with make(y=0, method='add'):
                make('termizator', size=6, block_num=12)
                make('pouring_name', size=total_size - 6, name='чильеджина  3,6 альче  8500кг')
            with make(y=1, method='add'):
                make('pouring_and_fermenting', size=8)
                make('soldification', size=7)
                make('cutting', size=7)
                make('pouring_off', size=3)
                make('extra', size=total_size - 8 - 8 - 7 - 3)

        make('drenator', size=cast_t('04:00'), visible=False)

        total_size = 31
        with make('melting', y=10, size=total_size, line='1'):
            with make(y=0, method='add'):
                make('melting_label', size=3)
                make('melting_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1, method='add'):
                make('serving', size=6)
                make('melting_process', size=total_size - 6, speed=900)
            with make(y=2, method='add'):
                make(size=6, visible=False)
                make('cooling1', size=5)
                make('cooling2', size=10)
        
        total_size=20
        with make('packing', size=total_size, line='1', t=27 + 6 + 5 + 10 + cast_t('04:00'), method='add'):
            with make(y=0, method='add'):
                make('packing_label', size=3)
                make('packing_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1, method='add'):
                make('packing_brand', size=total_size, name='безлактозная/претто')
            with make(y=2, method='add'):
                make('configuration', size=3, visible=False)
                make('configuration', size=1)
                make('configuration', size=5, visible=False)
                make('configuration', size=1)
                
    return fetch_constructed_blocks()[0]

In [428]:
root = Block('root', t=cast_t('01:00'), index_width=4)
boiling1 = make_boiling('1', '1')
boiling2 = make_boiling('2', '2')
root.push(boiling1)
root.push(boiling2, validator=atom_validator)
root

root None
  boiling ()
    pouring (12,39]
      block (12,39]
        termizator (12,18]
        pouring_name (18,39]
      block (12,39]
        pouring_and_fermenting (12,20]
        soldification (20,27]
        cutting (27,34]
        pouring_off (34,37]
        extra (37,38]
    packing (108,128]
      block (108,128]
        packing_label (108,111]
        packing_name (111,128]
      block (108,128]
        packing_brand (108,128]
      block (108,128]
        configuration (108,111]
        configuration (111,112]
        configuration (112,117]
        configuration (117,118]

In [307]:
clock.stats()

,atom_validate,atom_validator
count,8.000000,2.000000
mean,0.000014,0.000081
sum,0.000113,0.000162
max,0.000036,0.000106
min,0.000005,0.000056


In [240]:
clock.reset()
for i in range(100):
    try:
        atom_validator(boiling1, boiling2)
    except:
        pass

In [86]:
clock.stats()

,atom_validate,atom_validator
count,106.000000,1.000000
mean,0.000235,0.025246
sum,0.024867,0.025246
max,0.000448,0.025246
min,0.000078,0.025246


In [426]:
work_book, sheet = init_sheet()
# work_book, sheet = init_template_sheet()
# update styles
for b in root:
    if b['class'] in style:
        b.local_props.update(style[b['class']])

root.update_props()
    
draw(sheet, root)
work_book.save('output.xlsx')

termizator 1 (5,11]
pouring_name 1 (11,32]
pouring_and_fermenting 2 (5,13]
soldification 2 (13,20]
cutting 2 (20,27]
pouring_off 2 (27,30]
extra 2 (30,31]
melting_label 5 (8,11]
melting_name 5 (11,39]
serving 6 (8,14]
melting_process 6 (14,39]
cooling1 7 (14,19]
cooling2 7 (19,29]
packing_label 8 (29,32]
packing_name 8 (32,49]
packing_brand 9 (29,49]
configuration 10 (32,33]
configuration 10 (38,39]
